In [1]:
%pwd

'/home/hajibagher/insect-classification-deeplearning'

In [13]:
from pprint import pprint
import torch
from torch import nn
import torchviz

import vit
import scipy.io
from matplotlib import pyplot as plt
import numpy as np
import dnaencoder
import pandas as pd
from importlib import reload

In [2]:
from mat import mat

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
print("Device", device)

Device cuda


In [5]:
dict(dnaencoder.evaluate_model(mat, "amintehrani/dnaencoder-insects-finetuned"))

Evaluating model: amintehrani/dnaencoder-insects-finetuned


tokenizer_config.json:   0%|          | 0.00/958 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/101 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

esm_config.py: 0.00B [00:00, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/amintehrani/dnaencoder-insects-finetuned:
- esm_config.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_esm.py: 0.00B [00:00, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/amintehrani/dnaencoder-insects-finetuned:
- modeling_esm.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


KeyboardInterrupt: 

In [ ]:
model_name = "amintehrani/vit-insects-finetuned7"
dict(vit.evaluate_model(mat, model_name, device))

[('val_seen_indices',
  {'eval_loss': 0.82353675365448,
   'eval_model_preparation_time': 0.0026,
   'eval_accuracy': 0.8528138528138528,
   'eval_runtime': 16.6445,
   'eval_samples_per_second': 194.299,
   'eval_steps_per_second': 6.128}),
 ('val_unseen_indices',
  {'eval_loss': 10.643092155456543,
   'eval_model_preparation_time': 0.003,
   'eval_accuracy': 0.013705993012631014,
   'eval_runtime': 18.0735,
   'eval_samples_per_second': 205.882,
   'eval_steps_per_second': 6.474}),
 ('test_seen_indices',
  {'eval_loss': 2.636476755142212,
   'eval_model_preparation_time': 0.0029,
   'eval_accuracy': 0.6951903807615231,
   'eval_runtime': 24.294,
   'eval_samples_per_second': 205.401,
   'eval_steps_per_second': 6.421}),
 ('test_unseen_indices',
  {'eval_loss': 10.732027053833008,
   'eval_model_preparation_time': 0.0029,
   'eval_accuracy': 0.01599462365591398,
   'eval_runtime': 36.441,
   'eval_samples_per_second': 204.166,
   'eval_steps_per_second': 6.394})]

In [6]:
all_images = mat['all_images']
all_labels = mat['all_labels'].squeeze()
train_indices = mat['train_loc'].squeeze()
val_seen_indices = mat['val_seen_loc'].squeeze()
val_unseen_indices = mat['val_unseen_loc'].squeeze()
test_seen_indices = mat['test_seen_loc'].squeeze()
test_unseen_indices = mat['test_unseen_loc'].squeeze()

val_indices = np.concatenate((val_seen_indices, val_unseen_indices))
test_indices = np.concatenate((test_seen_indices, test_unseen_indices))

total_images = all_images.shape[0]
print(train_indices.shape[0], val_seen_indices.shape[0], val_unseen_indices.shape[0], test_seen_indices.shape[0], test_unseen_indices.shape[0])



13039 3234 3721 4990 7440


In [7]:
mat["val_seen_loc"].shape

(1, 3234)

In [8]:
from multimodal_dataset import MultiModalDataset
from load_embeddings import load_dna_embeddings, load_img_embeddings

all_dna_features = load_dna_embeddings()
all_image_features = load_img_embeddings()

all_dna_len = list(map(lambda s: len(s.strip()), mat['all_string_dnas']))
dna_str_len_mapping: dict[int,int] = {}

def dna_str_len_to_int(s_len):
    if s_len not in dna_str_len_mapping:
        dna_str_len_mapping[s_len] = len(dna_str_len_mapping)
    return dna_str_len_mapping[s_len]

# def all_dna_len_token():
#     return list(map(dna_str_len_to_int, all_dna_len))

all_dna_len_tokens = list(map(dna_str_len_to_int, all_dna_len))
all_dna_len_tokens = np.array(all_dna_len_tokens, dtype=np.int64)

species2genus = mat['species2genus']-1


genus_species = dict()
max_specie_in_genus = 0
for genus_id, genus in pd.DataFrame(species2genus, columns=['genus']).groupby('genus'):
    specie_indices = genus.index.tolist()
    genus_species[genus_id] = specie_indices
    if len(specie_indices) > max_specie_in_genus:
        max_specie_in_genus = len(specie_indices)


datasets = {
    indicies_type:MultiModalDataset(
        mat['all_string_dnas'][(inds:=mat[indicies_type].flatten())],
        mat['all_images'][inds],
        np.transpose(mat['all_labels'], (1,0))[inds],
        dna_str_len_mapping,
        species2genus,
        genus_species,
        None,
        None,
        dna_embeddings=all_dna_features[inds],
        img_embeddings=all_image_features[inds]
    )
    for indicies_type in ["val_seen_loc", "val_unseen_loc", "test_seen_loc", "test_unseen_loc"]
}
datasets

Loading file: batch_dna_0.pkl
Loading file: batch_dna_1.pkl
Loading file: batch_dna_2.pkl
Loading file: batch_dna_3.pkl
Loading file: batch_dna_4.pkl
Loading file: batch_dna_5.pkl
Loading file: batch_dna_6.pkl
Loading file: batch_dna_7.pkl
Loading file: batch_dna_8.pkl
Loading file: batch_dna_9.pkl
Loading file: batch_dna_10.pkl
Loading file: batch_dna_11.pkl
Loading file: batch_dna_12.pkl
Loading file: batch_dna_13.pkl
Loading file: batch_dna_14.pkl
Loading file: batch_dna_15.pkl
Loading file: batch_dna_16.pkl
Loading file: batch_dna_17.pkl
Loading file: batch_dna_18.pkl
Loading file: batch_dna_19.pkl
Loading file: batch_dna_20.pkl
Loading file: batch_dna_21.pkl
Loading file: batch_dna_22.pkl
Loading file: batch_dna_23.pkl
Loading file: batch_dna_24.pkl
Loading file: batch_dna_25.pkl
Loading file: batch_dna_26.pkl
Loading file: batch_dna_27.pkl
Loading file: batch_dna_28.pkl
Loading file: batch_dna_29.pkl
Loading file: batch_dna_30.pkl
Loading file: batch_dna_31.pkl
Loading file: batc

{'val_seen_loc': <multimodal_dataset.MultiModalDataset at 0x7fd5b620fc10>,
 'val_unseen_loc': <multimodal_dataset.MultiModalDataset at 0x7fd5b5ee72d0>,
 'test_seen_loc': <multimodal_dataset.MultiModalDataset at 0x7fd6c33c63d0>,
 'test_unseen_loc': <multimodal_dataset.MultiModalDataset at 0x7fd6b2b4c190>}

In [9]:
dict(main_classifier.evaluate(datasets))

NameError: name 'main_classifier' is not defined

In [29]:
# Evaluate:
import models
reload(models)
from models import AttentionFusion, GenusClassifier, LocalSpecieClassfier, MainClassifier, multimodal_collector

fusion_embedder = AttentionFusion(dna_dim=512,img_dim=768,dna_len_dim=32, fused_dim=258, proj_dna_dim=128-32, proj_img_dim=128, dropout=0.2)
# print("fusion_embedder", fusion_embedder._config)
# print("Fusion model created. fused dim: ", fusion_embedder.fused_dim)
genus_classifier = GenusClassifier(fusion_embedder.fused_dim,dropout=0.1, dna_len_dim=32)
local_specie_classifier = LocalSpecieClassfier(fusion_embedder.fused_dim,reduced_fused_dim=128, specie_decoder_hidden_dim=256, dropout=0.1,dna_len_dim=32)

# main_classifier = MainClassifier(mat['species2genus'], genus_species, None, None, fusion_embedder, genus_classifier,local_specie_classifier,alpha=2, beta=0, theta=0,).to(device)
main_classifier = MainClassifier.load_model("./results_20250827T234918-final", mat['species2genus'], genus_species).to(device)

assert main_classifier.fusion_embedder._config == fusion_embedder._config
assert main_classifier.genus_classifier._config == genus_classifier._config
assert main_classifier.local_specie_classifier._config == local_specie_classifier._config
# print(main_classifier)
dict(main_classifier.evaluate(datasets))


# print(main_classifier)
# main_classifier.load_model("results_20250819T030148-final")
# train_indices = (mat['train_loc'] - 1).flatten()  # Get train indices
# train_dataset = MultiModalDataset(mat['all_string_dnas'][train_indices], mat['all_images'][train_indices], np.transpose(mat['all_labels'], (1,0))[train_indices], dna_str_len_mapping, species2genus, genus_species, None, None, dna_embeddings=all_dna_features[train_indices], img_embeddings=all_image_features[train_indices])
# main_classifier.fit(
#     train_dataset,
#     None,
#     batch_size=64,
#     epochs=1,
#     eval_steps=200,
#     save_steps=400,
#     lr=1e-3
# )

Loaded model config: {'fusion_embedder': {'dna_dim': 512, 'img_dim': 768, 'fused_dim': 258, 'dna_len_dim': 32, 'num_distinct_dna_len': 120, 'proj_dna_dim': 96, 'proj_img_dim': 128, 'dropout': 0.2}, 'genus_classifier': {'fused_dim': 258, 'hidden_dim': 744, 'genus_n_classes': 372, 'dropout': 0.1, 'dna_len_dim': 32}, 'local_specie_classifier': {'fused_dim': 258, 'genus_n_classes': 372, 'reduced_fused_dim': 128, 'max_specie_in_genus': 23, 'genus_embedding_dim': 64, 'specie_decoder_hidden_dim': 256, 'dropout': 0.1, 'dna_len_dim': 32}}
Loaded model state: <All keys matched successfully>
Evaluating val_seen_loc...


/home/hajibagher/insect-classification-deeplearning/multimodal_dataset.py:88: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  res['dna_emb'] = torch.tensor(self.dna_embeddings[idx], dtype=torch.float32)
/home/hajibagher/insect-classification-deeplearning/multimodal_dataset.py:92: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  res['img_emb'] = torch.tensor(self.img_embeddings[idx], dtype=torch.float32)
/opt/conda/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Evaluating val_unseen_loc...


/home/hajibagher/insect-classification-deeplearning/multimodal_dataset.py:88: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  res['dna_emb'] = torch.tensor(self.dna_embeddings[idx], dtype=torch.float32)
/home/hajibagher/insect-classification-deeplearning/multimodal_dataset.py:92: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  res['img_emb'] = torch.tensor(self.img_embeddings[idx], dtype=torch.float32)
/opt/conda/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Evaluating test_seen_loc...


/home/hajibagher/insect-classification-deeplearning/multimodal_dataset.py:88: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  res['dna_emb'] = torch.tensor(self.dna_embeddings[idx], dtype=torch.float32)
/home/hajibagher/insect-classification-deeplearning/multimodal_dataset.py:92: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  res['img_emb'] = torch.tensor(self.img_embeddings[idx], dtype=torch.float32)
/opt/conda/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Evaluating test_unseen_loc...


/home/hajibagher/insect-classification-deeplearning/multimodal_dataset.py:88: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  res['dna_emb'] = torch.tensor(self.dna_embeddings[idx], dtype=torch.float32)
/home/hajibagher/insect-classification-deeplearning/multimodal_dataset.py:92: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  res['img_emb'] = torch.tensor(self.img_embeddings[idx], dtype=torch.float32)
/opt/conda/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


{'val_seen_loc': {'eval_loss': 15.198629379272461,
  'eval_accuracy': 0.5355596784168213,
  'eval_runtime': 2.1439,
  'eval_samples_per_second': 1508.435,
  'eval_steps_per_second': 23.788},
 'val_unseen_loc': {'eval_loss': 19.26132583618164,
  'eval_accuracy': 0.10131685030905671,
  'eval_runtime': 2.4401,
  'eval_samples_per_second': 1524.945,
  'eval_steps_per_second': 24.179},
 'test_seen_loc': {'eval_loss': 15.887377738952637,
  'eval_accuracy': 0.43967935871743485,
  'eval_runtime': 3.1852,
  'eval_samples_per_second': 1566.637,
  'eval_steps_per_second': 24.489},
 'test_unseen_loc': {'eval_loss': 19.262758255004883,
  'eval_accuracy': 0.12352150537634408,
  'eval_runtime': 4.7665,
  'eval_samples_per_second': 1560.893,
  'eval_steps_per_second': 24.546}}

In [ ]:
main_classifier = MainClassifier(mat['species2genus'], genus_species, None, None, None, None,local_specie_classifier,alpha=2, beta=0, theta=0,).to(device)

In [ ]:
device

device(type='cuda')

In [15]:

main_classifier2 = MainClassifier.load_modelV2("./model_trained_v2_20250828T164138-final", mat['species2genus'], genus_species).to(device)

pprint(dict(main_classifier2.evaluate(datasets)))

Loaded model config: {'fusion_embedder': {'num_distinct_dna_len': 120, 'dna_len_dim': 32, 'fused_dim': 256, 'n_heads': 4, 'ff_mult': 4, 'dropout': 0.2, 'num_layers': 1}, 'genus_classifier': {'fused_dim': 256, 'hidden_dim': 744, 'genus_n_classes': 372, 'dropout': 0.2, 'dna_len_dim': 32}, 'local_specie_classifier': {'fused_dim': 256, 'genus_n_classes': 372, 'reduced_fused_dim': 128, 'max_specie_in_genus': 23, 'genus_embedding_dim': 64, 'specie_decoder_hidden_dim': 256, 'dropout': 0.2, 'dna_len_dim': 32}}
Loaded model state: <All keys matched successfully>
Evaluating val_seen_loc...


/home/hajibagher/insect-classification-deeplearning/multimodal_dataset.py:88: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  res['dna_emb'] = torch.tensor(self.dna_embeddings[idx], dtype=torch.float32)
/home/hajibagher/insect-classification-deeplearning/multimodal_dataset.py:92: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  res['img_emb'] = torch.tensor(self.img_embeddings[idx], dtype=torch.float32)
/opt/conda/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Evaluating val_unseen_loc...


/home/hajibagher/insect-classification-deeplearning/multimodal_dataset.py:88: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  res['dna_emb'] = torch.tensor(self.dna_embeddings[idx], dtype=torch.float32)
/home/hajibagher/insect-classification-deeplearning/multimodal_dataset.py:92: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  res['img_emb'] = torch.tensor(self.img_embeddings[idx], dtype=torch.float32)
/opt/conda/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Evaluating test_seen_loc...


/home/hajibagher/insect-classification-deeplearning/multimodal_dataset.py:88: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  res['dna_emb'] = torch.tensor(self.dna_embeddings[idx], dtype=torch.float32)
/home/hajibagher/insect-classification-deeplearning/multimodal_dataset.py:92: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  res['img_emb'] = torch.tensor(self.img_embeddings[idx], dtype=torch.float32)
/opt/conda/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Evaluating test_unseen_loc...


/home/hajibagher/insect-classification-deeplearning/multimodal_dataset.py:88: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  res['dna_emb'] = torch.tensor(self.dna_embeddings[idx], dtype=torch.float32)
/home/hajibagher/insect-classification-deeplearning/multimodal_dataset.py:92: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  res['img_emb'] = torch.tensor(self.img_embeddings[idx], dtype=torch.float32)
/opt/conda/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


{'test_seen_loc': {'eval_accuracy': 0.7845691382765531,
                   'eval_loss': 20.35539436340332,
                   'eval_runtime': 2.448,
                   'eval_samples_per_second': 2038.365,
                   'eval_steps_per_second': 15.931},
 'test_unseen_loc': {'eval_accuracy': 0.11922043010752688,
                     'eval_loss': 32.198673248291016,
                     'eval_runtime': 3.6753,
                     'eval_samples_per_second': 2024.313,
                     'eval_steps_per_second': 16.053},
 'val_seen_loc': {'eval_accuracy': 0.9329004329004329,
                  'eval_loss': 17.840614318847656,
                  'eval_runtime': 1.8435,
                  'eval_samples_per_second': 1754.29,
                  'eval_steps_per_second': 14.104},
 'val_unseen_loc': {'eval_accuracy': 0.11341037355549584,
                    'eval_loss': 32.35670471191406,
                    'eval_runtime': 1.8621,
                    'eval_samples_per_second': 1998.331,
      

In [14]:
main_classifier2 = MainClassifier.load_model("./model_trained_20250828T164930-final", mat['species2genus'], genus_species).to(device)

pprint(dict(main_classifier2.evaluate(datasets)))

Loaded model config: {'fusion_embedder': {'dna_dim': 512, 'img_dim': 768, 'fused_dim': 256, 'dna_len_dim': 32, 'num_distinct_dna_len': 120, 'proj_dna_dim': 96, 'proj_img_dim': 128, 'dropout': 0.2}, 'genus_classifier': {'fused_dim': 256, 'hidden_dim': 744, 'genus_n_classes': 372, 'dropout': 0.2, 'dna_len_dim': 32}, 'local_specie_classifier': {'fused_dim': 256, 'genus_n_classes': 372, 'reduced_fused_dim': 128, 'max_specie_in_genus': 23, 'genus_embedding_dim': 64, 'specie_decoder_hidden_dim': 256, 'dropout': 0.2, 'dna_len_dim': 32}}
Loaded model state: <All keys matched successfully>
Evaluating val_seen_loc...


/home/hajibagher/insect-classification-deeplearning/multimodal_dataset.py:88: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  res['dna_emb'] = torch.tensor(self.dna_embeddings[idx], dtype=torch.float32)
/home/hajibagher/insect-classification-deeplearning/multimodal_dataset.py:92: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  res['img_emb'] = torch.tensor(self.img_embeddings[idx], dtype=torch.float32)
/opt/conda/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Evaluating val_unseen_loc...


/home/hajibagher/insect-classification-deeplearning/multimodal_dataset.py:88: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  res['dna_emb'] = torch.tensor(self.dna_embeddings[idx], dtype=torch.float32)
/home/hajibagher/insect-classification-deeplearning/multimodal_dataset.py:92: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  res['img_emb'] = torch.tensor(self.img_embeddings[idx], dtype=torch.float32)
/opt/conda/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Evaluating test_seen_loc...


/home/hajibagher/insect-classification-deeplearning/multimodal_dataset.py:88: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  res['dna_emb'] = torch.tensor(self.dna_embeddings[idx], dtype=torch.float32)
/home/hajibagher/insect-classification-deeplearning/multimodal_dataset.py:92: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  res['img_emb'] = torch.tensor(self.img_embeddings[idx], dtype=torch.float32)
/opt/conda/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Evaluating test_unseen_loc...


/home/hajibagher/insect-classification-deeplearning/multimodal_dataset.py:88: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  res['dna_emb'] = torch.tensor(self.dna_embeddings[idx], dtype=torch.float32)
/home/hajibagher/insect-classification-deeplearning/multimodal_dataset.py:92: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  res['img_emb'] = torch.tensor(self.img_embeddings[idx], dtype=torch.float32)
/opt/conda/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


{'test_seen_loc': {'eval_accuracy': 0.7707414829659318,
                   'eval_loss': 20.461692810058594,
                   'eval_runtime': 2.4174,
                   'eval_samples_per_second': 2064.229,
                   'eval_steps_per_second': 16.133},
 'test_unseen_loc': {'eval_accuracy': 0.10188172043010753,
                     'eval_loss': 32.160709381103516,
                     'eval_runtime': 3.3767,
                     'eval_samples_per_second': 2203.326,
                     'eval_steps_per_second': 17.473},
 'val_seen_loc': {'eval_accuracy': 0.91713048855906,
                  'eval_loss': 17.949588775634766,
                  'eval_runtime': 1.5887,
                  'eval_samples_per_second': 2035.688,
                  'eval_steps_per_second': 16.366},
 'val_unseen_loc': {'eval_accuracy': 0.07309862940069874,
                    'eval_loss': 32.578216552734375,
                    'eval_runtime': 1.8592,
                    'eval_samples_per_second': 2001.407,
    